# My Q\# notebook

> Superdense Coding and Deutsch-Josza were done by completing the Quanta Katas by [Microsoft](https://github.com/microsoft/QuantumKatas).

## Table of Contents
* [Superdense Coding](#Superdense-Coding)
* [Deutsch-Josza Algorithm](#Deutsch-Josza)
* [Writing Circuit Models](#Circuit-Models)
* [Quantum Fourier Transforms](#QFT)

---

## Superdense Coding <a class="anchor" id="Superdense-Coding"></a>

We start with some superdense coding. Check 1.3.11 from my notes.

In [ ]:
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Measurement;

newtype ProtocolMessage = (Bit1 : Bool, Bit2 : Bool);

operation CreateEntangledPair (q1 : Qubit, q2 : Qubit) : Unit is Adj {
    H(q1);
    CNOT(q1,q2);
}

operation EncodeMessageInQubit (qAlice : Qubit, message : ProtocolMessage) : Unit {
    if (message::Bit1) == false {
        if (message::Bit2) == true {
            X(qAlice);
        }
    } else {
        Z(qAlice);
        if (message::Bit2) == true {
            X(qAlice);
        }
    }
}

operation DecodeMessageFromQubits (qAlice : Qubit, qBob : Qubit) : ProtocolMessage {
    CNOT(qAlice,qBob);
    H(qAlice);
    mutable bit1 = false;
    mutable bit2 = false;
    if M(qAlice) == One {
        set bit1 = true;
    }
    if M(qBob) == One {
        set bit2 = true;
    }
    let mes = ProtocolMessage(bit1,bit2);
    ResetAll([qAlice,qBob]);
    return mes;
}

operation SuperdenseCodingProtocol (message : ProtocolMessage) : ProtocolMessage {
    use q = Qubit[2];
    CreateEntangledPair(q[0],q[1]);
    EncodeMessageInQubit(q[0],message);
    return DecodeMessageFromQubits(q[0],q[1]);
}

We now test our code by going through all permutations of $\{\text{true},\text{false}\}$

In [ ]:
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Diagnostics;

operation TestSuperDenseCoding () : Bool {
    for n in 0 .. 3 {
        let data = ProtocolMessage(1 == n / 2, 1 == n % 2);
        
        for iter in 1 .. 100 {
            let result = SuperdenseCodingProtocol(data);
            
            Fact(result::Bit1 == data::Bit1 and result::Bit2 == data::Bit2, 
                    $"{data} was transfered incorrectly as {result}");
        }
    }
    return true;
}

In [ ]:
%simulate TestSuperDenseCoding

---
## Deutsch-Josza <a class="anchor" id="Deutsch-Josza"></a>

In [ ]:
operation DeutschJozsaAlgorithm (N : Int, oracle : (Qubit[] => Unit)) : Bool {
    mutable isConstant = true;
    use x = Qubit[N];
    ApplyToEach(H,x);
    oracle(x);
    ApplyToEach(H,x);
    for q in 0 .. N-1 {
        if M(x[q]) != Zero {
            set isConstant = false; 
        }
    }
    return isConstant;
}

operation OracleZero (x : Qubit[]) : Unit {}

Let us test our operation

In [ ]:
operation TestDJ () : Bool {
    Fact(DeutschJozsaAlgorithm(4, OracleZero) == true, "f(x) = 0 not identified as constant");
    return true;
}

In [ ]:
%simulate TestDJ

---
## Writing Circuit Models <a class="anchor" id="Circuit-Models"></a>

The following is my implementation for Question 8, Chapter 2 from de Wolf's *Quantum Computing* lecture notes (or 1.4.7 from my notes).

In [ ]:
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Diagnostics;
    
operation CheckIfAllZeros (q : Qubit[], b : Qubit) : Unit {
    // we start with using N auxiliary qubits -- where N is the number of how many qubits there are in q
    let N = Length(q);
    use d = Qubit[N];
    
    // we now apply the X gate to all qubits in q
    ApplyToEach(X,q);
    
    // if N = 1, then we apply a CNOT gate directly from q to d.
    // otherwise, we start with applying a CCNOT gate with the first two qubits in q 
    // being control and the first qubit in d being the target.
    if N > 1 {
        CCNOT(q[0],q[1],d[0]);
    } else {
        CNOT(q[0],d[0]);
    }
    
    // if N >= 3, then we apply a CCNOT looped over the range of N - 3, where we have
    // our control qubits being q[i+2] and d[i] and our target being d[i+1]
    for i in 0 .. N - 3 {
        CCNOT(q[i+2],d[i],d[i+1]);
    }
    
    // we now apply a CCNOT gate to our last qubit in d with the two before it being control qubits
    if N > 2 {
        CCNOT(d[N-3],d[N-2],d[N-1]);
    }
    
    // for the special case of N = 2, we apply a CNOT gate from the first qubit in d to the last qubit in d
    if N == 2 {
        CNOT(d[0],d[1]);
    }
    
    // this step is the step that either changes b to 1-b (if all q is zero) or keeps it as b (not all q is zero).
    // we do this by apply a CNOT gate from the last qubit in d to b
    CNOT(d[N-1],b);
    
    // we now return all our qubits to their original states (except for b)
    ApplyToEach(X,q);
    ResetAll(d);
}

The following code is not necessary to run. This is just to have our output in bitstrings rather than the default.

In [ ]:
%config dump.basisStateLabelingConvention = "Bitstring"

We now test our operation with the following code. Click run on %simulate to see the output. Feel free to edit the test operation.

> Remember to run CheckIfAllZeros operation and the TestCheckIfAllZeros operation before running the %simulation field.
> Uncomment the `DumpMachine();` lines to see the output of the qubits.

In [ ]:
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Diagnostics;

operation TestCheckIfAllZeros () : Bool {
    use (qs, b) = (Qubit[2], Qubit());
    // X(qs[3]);
    // X(b);
    
    //DumpMachine();
    CheckIfAllZeros(qs,b);
    //DumpMachine();
    
    ResetAll(qs);
    return M(b) == One;
}

In [ ]:
%simulate TestCheckIfAllZeros

Run the following code to see the circuit diagram Q\# draws. The first $N$ qubits are $|q\rangle$, the $(N+1)^\text{th}$ qubit is $|b\rangle$, and everything that follows is an auxiliary qubit.

In [ ]:
%trace TestCheckIfAllZeros

---
## Quantum Fourier Transforms <a class="anchor" id="QFT"></a>

In [1]:
operation my_QFT (qs : Qubit[]) : Unit is Adj+Ctl {
    let N = Length(qs);
    
    for i in 0 .. N-1 {
        H(qs[i]);
        for j in 1 .. N-i-1 {
            Controlled R1Frac([qs[j+i]], (2, j+1, qs[i]));
        }
    }
    
    for i in 0 .. N/2 - 1 {
        SWAP(qs[i], qs[N-1-i]);
    }
}

my_QFT

We can verify our `QFT` operation by ensuring that it is equal to the one defined in the library.
> Since the library's `QFT` operation is defined as `BigEndian` $\to$ `Unit is Adj+Ctl`, we define the `lib_QFT` operation which takes in `Qubit[]` and outputs `Unit is Adj+Ctl` by converting the input to `BigEndian` before applying it to the library's `QFT`.

In [2]:
open Microsoft.Quantum.Bitwise;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Preparation;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Math;

operation lib_QFT (qs : Qubit[]) : Unit is Adj+Ctl {
    QFT(BigEndian(qs));
}

operation Test_QFT () : Unit {
    for i in 1 .. 10 {
        AssertOperationsEqualReferenced(i, my_QFT, lib_QFT);
    }
    Message("lib_QFT = my_QFT!");
}

lib_QFT, Test_QFT

In [3]:
%simulate Test_QFT

lib_QFT = my_QFT!


()

What happens when the quantum Fourier transform is applied?

In [ ]:
operation output_my_QFT () : Unit {
    use qs = Qubit[4];
    X(qs[0]);
    X(qs[3]);
    Z(qs[0]);
    DumpMachine();
    my_QFT(qs);
    DumpMachine();
    ResetAll(qs);
}

In [ ]:
%simulate output_my_QFT

We generally work with `LittleEndian` so then we have the following code.

In [20]:
operation my_little_endian_QFT (qs : Qubit[]) : Unit is Adj+Ctl {
    for i in 0 .. Length(qs)/2 - 1 {
        SWAP(qs[i], qs[Length(qs)-1-i]);
    }
    my_QFT(qs);
    for i in 0 .. Length(qs)/2 - 1 {
        SWAP(qs[i], qs[Length(qs)-1-i]);
    }
}

my_little_endian_QFT

In [23]:
operation lib_le_QFT (qs : Qubit[]) : Unit is Adj+Ctl {
    QFTLE(LittleEndian(qs));
}

lib_le_QFT

In [25]:
operation Test_LE_QFT () : Unit {
    for i in 1 .. 10 {
        AssertOperationsEqualReferenced(i, my_little_endian_QFT, lib_le_QFT);
    }
    Message("lib_le_QFT = my_little_endian_QFT!");
}

Test_LE_QFT

In [26]:
%simulate Test_LE_QFT

lib_le_QFT = my_little_endian_QFT!


()

Now that we have verified that it is equal to the library's little endian version of the quantum Fourier transform, we want to see if we have the same matrix as the one we defined in the notes. QSharp has an operation called `DumpOperation` that outputs the unitary representation.

In our notes we defined the QFT matrix as $F_N = \dfrac{1}{\sqrt{N}}\begin{pmatrix}&\vdots\\\cdots&\omega_N^{jk}&\cdots\\&\vdots\end{pmatrix}$, where $\omega_N=e^{2\pi{i}/N}$ and $N=2^n$ for $n$-qubits.

In [35]:
operation dump_my_little_QFT () : Unit {
    Message("F_2");
    DumpOperation(1, my_little_endian_QFT);
    Message("F_4");
    DumpOperation(2, my_little_endian_QFT);
    Message("F_8");
    DumpOperation(3, my_little_endian_QFT);
}

dump_my_little_QFT

In [36]:
%simulate dump_my_little_QFT

F_2


Real:
[[0.7071067811865476, 0.7071067811865476], 
[0.7071067811865476, -0.7071067811865476]]
Imag:
[[0, 0], 
[0, 0]]

F_4


Real:
[[0.5, 0.5, 0.5, 0.5], 
[0.5, 7.850462293418873E-17, -0.5, -7.850462293418873E-17], 
[0.5, -0.5, 0.5, -0.5], 
[0.5, -7.850462293418873E-17, -0.5, 7.850462293418873E-17]]
Imag:
[[0, 0, 0, 0], 
[0, 0.5, 0, -0.5], 
[0, 0, 0, 0], 
[0, -0.5, 0, 0.5]]

F_8


Real:
[[0.353553390593274, 0.353553390593274, 0.353553390593274, 0.353553390593274, 0.353553390593274, 0.353553390593274, 0.353553390593274, 0.353553390593274], 
[0.353553390593274, 0.2500000000000001, 7.850462293418881E-17, -0.2500000000000001, -0.353553390593274, -0.2500000000000001, -7.850462293418881E-17, 0.2500000000000001], 
[0.353553390593274, 5.551115123125785E-17, -0.353553390593274, -5.551115123125785E-17, 0.353553390593274, 5.551115123125785E-17, -0.353553390593274, -5.551115123125785E-17], 
[0.353553390593274, -0.2500000000000001, -7.850462293418881E-17, 0.2500000000000002, -0.353553390593274, 0.2500000000000001, 7.850462293418881E-17, -0.2500000000000002], 
[0.353553390593274, -0.353553390593274, 0.353553390593274, -0.353553390593274, 0.353553390593274, -0.353553390593274, 0.353553390593274, -0.353553390593274], 
[0.353553390593274, -0.2500000000000001, 7.850462293418881E-17, 0.2500000000000001, -0.353553390593274, 0.2500000000000001, -7.850462293418881E-17, -0.2500000000000001], 
[0.353553390593274, -5.551115123125785E-17, -0.353553390593274, 5.551115123125785E-17, 0.353553390593274, -5.551115123125785E-17, -0.353553390593274, 5.551115123125785E-17], 
[0.353553390593274, 0.2500000000000001, -7.850462293418881E-17, -0.2500000000000002, -0.353553390593274, -0.2500000000000001, 7.850462293418881E-17, 0.2500000000000002]]
Imag:
[[7.156233269733757E-34, 7.156233269733757E-34, 7.156233269733757E-34, 7.156233269733757E-34, 7.156233269733757E-34, 7.156233269733757E-34, 7.156233269733757E-34, 7.156233269733757E-34], 
[7.156233269733757E-34, 0.25000000000000017, 0.353553390593274, 0.25000000000000017, -7.156233269733757E-34, -0.25000000000000017, -0.353553390593274, -0.25000000000000017], 
[7.156233269733757E-34, 0.353553390593274, -7.156233269733757E-34, -0.353553390593274, 7.156233269733757E-34, 0.353553390593274, -7.156233269733757E-34, -0.353553390593274], 
[7.156233269733757E-34, 0.25000000000000017, -0.353553390593274, 0.25000000000000006, -7.156233269733757E-34, -0.25000000000000017, 0.353553390593274, -0.25000000000000006], 
[7.156233269733757E-34, -7.156233269733757E-34, 7.156233269733757E-34, -7.156233269733757E-34, 7.156233269733757E-34, -7.156233269733757E-34, 7.156233269733757E-34, -7.156233269733757E-34], 
[7.156233269733757E-34, -0.25000000000000017, 0.353553390593274, -0.25000000000000017, -7.156233269733757E-34, 0.25000000000000017, -0.353553390593274, 0.25000000000000017], 
[7.156233269733757E-34, -0.353553390593274, -7.156233269733757E-34, 0.353553390593274, 7.156233269733757E-34, -0.353553390593274, -7.156233269733757E-34, 0.353553390593274], 
[7.156233269733757E-34, -0.25000000000000017, -0.353553390593274, -0.25000000000000006, -7.156233269733757E-34, 0.25000000000000017, 0.353553390593274, 0.25000000000000006]]

()

This is clear that it is the same as our defined QFT.